In [ ]:
import warnings
warnings.filterwarnings('ignore')

# import keras
import pandas as pd
# import tensorflow as tf
import scipy.io as sio
import numpy as np
import itertools
import matplotlib.pyplot as plt
import os
import h5py

from numpy import array, mean, std, dstack,argmax
from pandas import read_csv
from matplotlib import pyplot

warnings.filterwarnings('ignore')

In [ ]:
Filename = '..'

In [ ]:
df= pd.read_csv("Filename",header=None)

In [ ]:
def normalize(X_Raw_Data):
    result = df4.copy()
    for feature_name in df4.columns:
        max_value = df4[feature_name].max()
        min_value = df4[feature_name].min()
        result[feature_name] = (df4[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
Features = normalize(X_Raw_Data)

In [ ]:
def windows(series,window_size,overlap):
    start = 0
    while start+window_size <= series.shape[0]:
        yield int(start), int(start + window_size)
        start += window_size - overlap
        
        
def segment_signal(series,window_size,overlap):
    windowed_series = np.array([])
    for k in range(series.shape[1]):
        print("Windowing Column %d"%(k))
        temp_series = np.array([])
        for (start, end) in windows(series,window_size,overlap):
            if start==0:
                temp_series = series[start:end,k].T
            else:
                temp_series = np.vstack([temp_series,series[start:end,k].T])
        if k==0:
            windowed_series = temp_series
        else:
            windowed_series = np.vstack([windowed_series,temp_series])
    windowed_series = windowed_series.reshape((k+1,int(windowed_series.shape[0]/(k+1)),window_size))
    return windowed_series

def segment_Activity_Label(label_series,window_size,overlap):
    windowed_labels = np.array([])
    for (start, end) in windows(label_series,window_size,overlap):

        if start==0:
            windowed_labels = max(label_series[start:end])[0]
        else:
            windowed_labels = np.vstack([windowed_labels,max(label_series[start:end])[0]])
    return windowed_labels  

def segment_Score_Label(Label_series,window_size,overlap):
    windowed_Labels = np.array([])
    for (start, end) in windows(Label_series,window_size,overlap):

        if start==0:
            windowed_Labels = np.mean(Label_series[start:end])#[[0]]
        else:
            windowed_Labels = np.vstack([windowed_Labels,np.mean(Label_series[start:end])])#[0]])
    return windowed_Labels

In [ ]:
window_size = 64
overlap = 32

In [ ]:
X_full_appended_temp = np.asarray(Features)
X_full_appended = segment_signal(X_full_appended_temp,window_size,overlap)
X_full_appended = X_full_appended.reshape(X_full_appended.shape[1],X_full_appended.shape[0],X_full_appended.shape[2])

Activity_Label_full_temp = np.asarray(Activity_Label)
Activity_Label_full_temp = np.squeeze(np.ndarray.tolist(Activity_Label_full_temp))
Activity_Label_full_temp = y_full_temp.reshape(len(Activity_Label_full_temp),1)
Activity_Label_full = segment_label(Activity_Label_full_temp,window_size,overlap)

Score_Label_full_temp = np.asarray(Score_Label)
Score_Label_full_temp = np.squeeze(np.ndarray.tolist(Score_Label_full_temp))
Score_Label_full_temp = y_full_temp.reshape(len(Score_Label_full_temp),1)
Score_Label_full = segment_Label(Score_Label_full_temp,window_size,overlap)

In [ ]:
Person = ['Player 1']
Limb = ['Palm'] #Palm, Hand, Right_Leg, Left_Leg

In [ ]:
np.save("../dataset/Preprocessed Data/"Person+"_"+Limb"_testing_X_data",X_full_appended)

In [ ]:
np.save("../dataset/Preprocessed Data/"Person+"_"+Limb"_testing_Activity_Label_data",Activity_Label_full)

In [ ]:
np.save("../dataset/Preprocessed Data/"Person+"_"+Limb"_testing_Score_Label_data",Score_Label_full)